<a href="https://colab.research.google.com/github/konradbachusz/tensorflow-notes/blob/master/Autoencoders_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA with matplotlib and Autoencoders

In [1]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.1.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python2.7/dist-packages/tensorflow-2.1.0.dist-info/*
    /usr/local/lib/python2.7/dist-packages/tensorflow/*
    /usr/local/lib/python2.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.1.0


In [2]:
!pip install tensorflow==1.5

     |████████████████████████████████| 44.4MB 96kB/s 
     |████████████████████████████████| 3.0MB 40.0MB/s 
     |████████████████████████████████| 890kB 38.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp27-none-any.whl size=107221 sha256=d751ed85c9576e37ef7894c15d6f648af7bb42167fa4f30a5f330d1541469e02
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0


In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf

### To plot pretty figures inline in the notebook

In [0]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.mlab import PCA

In [5]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)

1.5.0
1.16.4
0.24.2
2.2.4


### Read in stocks data from the directory data/stocks.csv

A link where this CSV file can be downloaded is here: https://goo.gl/PUiiQv

* This is data for the monthly closing price of 10 stocks over a 10 year period. 
* There are 12 * 10 = 120 rows in this dataset and 10 columns, one for each stock
* The original CSV files has a header row and an additional first column to hold the date - we'll leave these out when we convert this to a matrix to do a PCA

In [6]:
prices = pd.read_csv('stocks.csv')
prices.head()

,Date,AAPL,ADBE,CVX,GOOG,IBM,MDLZ,MSFT,NFLX,ORCL,SBUX
0,3-Jan-17,121.104935,113.820000,110.759811,823.830017,174.420883,44.430000,64.737526,141.220001,40.230000,55.649239
1,1-Dec-16,115.320023,102.949997,116.584061,792.450012,164.687836,44.330002,61.765545,123.800003,38.299999,55.270943
2,1-Nov-16,110.042900,102.809998,110.502274,775.880005,160.947403,41.066032,59.896873,117.000000,40.033211,57.709953
3,3-Oct-16,112.472404,107.510002,102.728424,809.900024,151.113403,44.750420,59.159267,124.870003,38.270115,52.588333
4,1-Sep-16,111.987015,108.540001,100.933739,804.059998,156.186905,43.714809,56.868721,98.550003,38.975769,53.648621


### Sort the prices data by date

* We're going to work with stock returns and not prices because they tend to be stationery when calculated across years (returns from 2007 can be compared to returns from 2016 even of the stock prices may have changed)
* Returns are the percentage change in the price of the stock from the previous close
* Sort the prices by dates in preparation for calculating returns
* We'll work with just 3 inputs i.e. 3 stocks AAPL, GOOG and NFLX. Try this same code out with more stocks as practice!

In [7]:
prices['Date'] = pd.to_datetime(prices['Date'], infer_datetime_format=True)
prices = prices.sort_values(['Date'], ascending=[True])

prices = prices[['AAPL', 'GOOG', 'NFLX']]
prices.head()

,AAPL,GOOG,NFLX
120,11.107141,251.001007,3.258571
119,10.962033,224.949951,3.218571
118,12.037377,229.309311,3.312857
117,12.930043,235.925919,3.167143
116,15.701322,249.204208,3.128572


### Calculate returns

* The pct_change() function in pandas does this automatically for you, apply this only to numeric columns
* Leave out the first row (with the earliest date) because no returns will be calculated for it

In [8]:
returns = prices[[key for key in dict(prices.dtypes) if dict(prices.dtypes)[key] in ['float64', 'int64']]].pct_change()
returns = returns[1:]
returns.head()

,GOOG,AAPL,NFLX
119,-0.103789,-0.013064,-0.012275
118,0.019379,0.098097,0.029294
117,0.028855,0.074158,-0.043984
116,0.056282,0.214329,-0.012178
115,0.049788,0.007014,-0.114612


### Choose a small part of this dataset

*In order to make it easier to understand and follow the demo we choose only 20 rows from the dataset.*

In [9]:
returns_arr = returns.as_matrix()[:20]
returns_arr.shape

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(20, 3)

### Scale all inputs

PCA works better if the mean and variance of each input component is the same. Each of our inputs can be scaled to achieve this. Scaling the data involves:

* Subtracting the mean for every point
* Dividing by the variance

Every point on the data set is now centered around 0 and has the same scale

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
returns_arr_scaled = scaler.fit_transform(returns_arr)

In [11]:
returns_arr_scaled

array([[-0.83085735, -0.26570419, -0.27988663],
       [ 0.19850401,  0.48183445,  0.05469668],
       [ 0.27769246,  0.32084764, -0.53510448],
       [ 0.50691119,  1.26346797, -0.27910722],
       [ 0.45264219, -0.13068243, -1.10356751],
       [-0.16651438,  0.35775408, -1.07769436],
       [ 0.12257604,  0.16512878, -0.04561865],
       [ 0.88031173,  0.55008778,  1.30278588],
       [ 2.09513584,  1.4206431 ,  2.03764936],
       [-0.1289488 , -0.45151408, -1.20580256],
       [ 0.01821392,  0.40746153,  1.04538738],
       [-1.50058206, -2.30718683, -0.62555009],
       [-1.34257977, -0.69154794,  1.87670064],
       [-0.50816289,  0.8161854 ,  0.60136016],
       [ 2.57561617,  1.24912054, -0.80129032],
       [ 0.20404444,  0.39430968, -0.58880849],
       [-0.8106076 , -0.93708285, -1.31840651],
       [-0.79963904, -0.51882766,  1.30701792],
       [-0.14797953,  0.26976548, -0.19411235],
       [-1.0957766 , -2.39406044, -0.17064887]])

### Run PCA using a standard Python library

* Set standardize to False because we have already scaled the data ourselves
* **fracs** gives us the proportion of variance of each principal component i.e. how much each component contributes to explaining the variance in the input

In [12]:
results = PCA(returns_arr_scaled, standardize=False)
results.fracs

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: MatplotlibDeprecationWarning: The PCA class was deprecated in version 2.2.
  """Entry point for launching an IPython kernel.


array([0.60781955, 0.32166829, 0.07051216])

### Projection of the original data in PCA space

This has all the principal components from the original data. **The original data can be completely reconstructed if we use all the principal components**

In [13]:
results.Y #principal components

array([[-0.81352622, -0.0344661 , -0.41981705],
       [ 0.48330724, -0.03463209, -0.19943605],
       [ 0.28541572, -0.61112976, -0.10701119],
       [ 1.15841885, -0.4941762 , -0.58742383],
       [-0.04531414, -1.17026147,  0.26126386],
       [-0.11741815, -1.02128333, -0.51022887],
       [ 0.18735121, -0.08798269, -0.03914118],
       [ 1.28672675,  0.98550887,  0.38458144],
       [ 2.88930345,  1.31359118,  0.69730893],
       [-0.68672399, -1.09413487,  0.07570745],
       [ 0.5432214 ,  0.9716301 , -0.14155166],
       [-2.77222082, -0.04182361,  0.52840606],
       [-0.94790014,  2.20656767, -0.18790612],
       [ 0.36740601,  0.65000011, -0.85354254],
       [ 2.42484146, -1.53196526,  0.78063647],
       [ 0.27415654, -0.64923202, -0.21713054],
       [-1.51319453, -0.98980136, -0.06277129],
       [-0.59450541,  1.50444085, -0.0110293 ],
       [ 0.0422457 , -0.16866673, -0.31955874],
       [-2.45159092,  0.29781672,  0.92864413]])

### The weight vector

This is the vector used to project the original data into PCA space. This can be used on the principal components to reconstruct the original data.

In [14]:
#Eugine vectors (weights) of the PCA
results.Wt

array([[ 0.67658261,  0.69753442,  0.23596969],
       [-0.25910276, -0.07444624,  0.96297638],
       [ 0.68927623, -0.71267348,  0.13036406]])

### Generating the original data from the principal components and the weight vector

In [15]:
#original scaled and standardized data
np.dot(results.Y, results.Wt)

array([[-0.83085735, -0.26570419, -0.27988663],
       [ 0.19850401,  0.48183445,  0.05469668],
       [ 0.27769246,  0.32084764, -0.53510448],
       [ 0.50691119,  1.26346797, -0.27910722],
       [ 0.45264219, -0.13068243, -1.10356751],
       [-0.16651438,  0.35775408, -1.07769436],
       [ 0.12257604,  0.16512878, -0.04561865],
       [ 0.88031173,  0.55008778,  1.30278588],
       [ 2.09513584,  1.4206431 ,  2.03764936],
       [-0.1289488 , -0.45151408, -1.20580256],
       [ 0.01821392,  0.40746153,  1.04538738],
       [-1.50058206, -2.30718683, -0.62555009],
       [-1.34257977, -0.69154794,  1.87670064],
       [-0.50816289,  0.8161854 ,  0.60136016],
       [ 2.57561617,  1.24912054, -0.80129032],
       [ 0.20404444,  0.39430968, -0.58880849],
       [-0.8106076 , -0.93708285, -1.31840651],
       [-0.79963904, -0.51882766,  1.30701792],
       [-0.14797953,  0.26976548, -0.19411235],
       [-1.0957766 , -2.39406044, -0.17064887]])

## Set up the TensorFlow code for an Autoencoder

* The number of inputs of the auto-encoder is equal to the number of outputs (number of columns in our input data)
* We have one hidden layer with 2 neurons
* Use an AdamOptimizer to minimize the loss

Notice that:

* The reconstruction loss of the output is the **mean-square error**
* Each layer has **no activation function** i.e is a linear layer

An Autoencoder set up this way simply **performs a PCA**

In [0]:
import tensorflow as tf

n_inputs = 3
n_hidden = 2  # codings
n_outputs = n_inputs

learning_rate = 0.01

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()

### Run the training data through the Autoencoder

* We do not use any labels for this training only the input data
* The output is the reconstructed value using only the 2 principal components

*Note that the reconstruction will not be equal to the input data*

In [17]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})

    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[-0.5414952  -0.5649018  -0.22512446]
 [ 0.33594507  0.3396814   0.08071659]
 [ 0.35142988  0.24456729 -0.52112836]
 [ 0.9117743   0.8448002  -0.2025118 ]
 [ 0.27254063  0.05550228 -1.1375949 ]
 [ 0.1851571  -0.00588278 -1.011147  ]
 [ 0.14953355  0.13721713 -0.04049222]
 [ 0.6151921   0.8241359   1.2526579 ]
 [ 1.6144387   1.9175441   1.9467356 ]
 [-0.18114196 -0.39756322 -1.2156342 ]
 [ 0.11575574  0.30655745  1.0638553 ]
 [-1.8647817  -1.930589   -0.694383  ]
 [-1.2130667  -0.8254739   1.9012191 ]
 [ 0.08014021  0.20786774  0.7126493 ]
 [ 2.03749     1.8054208  -0.9030472 ]
 [ 0.3536847   0.23955046 -0.5604765 ]
 [-0.7673394  -0.98181283 -1.3101782 ]
 [-0.79204786 -0.5267004   1.3084785 ]
 [ 0.07226527  0.04200972 -0.15242776]
 [-1.7358555  -1.7322284  -0.2916632 ]]


### The output of the PCA is equal to the reconstructed output if we used only 2 principal components

In [18]:
np.dot(results.Y[:,[0,1]], results.Wt[[0,1]])

array([[-0.54148743, -0.56489667, -0.22515757],
       [ 0.33597054,  0.33970166,  0.08069597],
       [ 0.35145272,  0.2445836 , -0.52115407],
       [ 0.91180847,  0.84482659, -0.20252826],
       [ 0.27255922,  0.0555134 , -1.13762692],
       [ 0.18517425, -0.0058725 , -1.01117886],
       [ 0.14955513,  0.1372339 , -0.04051605],
       [ 0.61522888,  0.82416877,  1.25265028],
       [ 1.61449737,  1.91759669,  1.94674534],
       [-0.18113214, -0.39755939, -1.21567209],
       [ 0.11578211,  0.30658142,  1.06384063],
       [-1.8647998 , -1.93060584, -0.69443525],
       [-1.21306054, -0.82546365,  1.90119685],
       [ 0.0801637 ,  0.20788828,  0.71263144],
       [ 2.037542  ,  1.80545945, -0.90305726],
       [ 0.35370736,  0.23956651, -0.56050247],
       [-0.76734084, -0.98181828, -1.31022339],
       [-0.7920368 , -0.52668796,  1.30845574],
       [ 0.07228472,  0.04202443, -0.15245337],
       [-1.73586893, -1.73224039, -0.29171069]])